### 개요
멀티모달 RAG 시스템 구현 -> PDF에서 텍스트와 이미지를 추출하고, 이를 멀티모달 RAG 시스템을 통해 요약 및 검색하여 질문 응답을 제공

### 주요 구성 요소
- **PyMuPDF**: PDF에서 텍스트와 이미지를 추출하는 데 사용
- **Gemini 1.5-flash 모델**: 이미지와 표를 요약하는 데 사용
- **Cohere Embeddings**: 문서의 청크를 임베딩하는 데 사용
- **Chroma Vectorstore**: 문서 임베딩을 저장하고 검색하는 벡터 스토어
- **LangChain**: 검색 및 생성 파이프라인을 조정

### 다이어그램
   <img src="../images/multi_model_rag_with_captioning.svg" alt="Reliable-RAG" width="300">

### 동기
복잡한 문서를 효율적으로 요약하여 멀티모달 데이터를 쉽게 검색하고 간결하게 응답을 제공하는 것이 이 프로젝트의 목표입니다.

### 방법 설명
1. **텍스트 및 이미지 추출**: PyMuPDF를 사용하여 PDF에서 텍스트와 이미지를 추출합니다.
2. **요약**: 추출된 이미지와 표를 Gemini 모델을 사용하여 요약합니다.
3. **임베딩 생성**: Cohere를 통해 생성된 임베딩을 Chroma에 저장합니다.
4. **검색**: 쿼리를 기반으로 유사도 기반 검색기를 사용하여 관련 섹션을 가져옵니다.

### 이점
- 복잡하고 멀티모달로 구성된 문서에서 간단한 검색과 요약이 가능합니다.
- 텍스트와 이미지 모두에 대한 Q&A 과정이 간소화됩니다.
- 다른 문서 유형으로 확장이 용이한 유연한 구조입니다.

### 구현
1. **문서 청크 분할**: 텍스트 스플리터를 사용해 문서를 청크 단위로 겹쳐서 분할합니다.
2. **요약된 텍스트 및 이미지 콘텐츠 벡터화**: 요약된 텍스트와 이미지 콘텐츠를 벡터로 저장합니다.
3. **쿼리 처리**: 관련 문서 세그먼트를 검색하고 간결한 답변을 생성하여 응답합니다.

### 요약
이 프로젝트는 멀티모달 문서의 처리와 검색을 가능하게 하여, 최신 LLM과 벡터 기반 검색 시스템을 결합함으로써 간결하고 관련성 높은 응답을 제공합니다.


### Imports

In [2]:
import fitz  # PyMuPDF
from PIL import Image
import io
import os
from dotenv import load_dotenv

import google.generativeai as genai
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_cohere import ChatCohere, CohereEmbeddings

load_dotenv()

/opt/anaconda3/envs/kmy_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
# ! pip install google-generativeai



### Download the "Attention is all you need" paper

In [2]:
! curl -o attention_is_all_you_need.pdf https://arxiv.org/pdf/1706.03762
! mv 1706.03762 attention_is_all_you_need.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2163k  100 2163k    0     0  1068k      0  0:00:02  0:00:02 --:--:-- 1068k
mv: 1706.03762: No such file or directory


In [24]:
# ! wget https://arxiv.org/pdf/1706.03762
# ! mv 1706.03762 attention_is_all_you_need.pdf
! ls | grep attention_is_all_you_need.pdf

# ! ls | x.pdf

attention_is_all_you_need.pdf


### Data Extraction

In [17]:
text_data = []
img_data = []

In [18]:
with fitz.open('y.pdf') as pdf_file:
    # 이미지를 저장할 디렉토리가 없으면 새로 생성
    if not os.path.exists("extracted_images"):
        os.makedirs("extracted_images")

    # PDF 파일의 각 페이지를 순회하며 처리
    for page_number in range(len(pdf_file)):
        page = pdf_file[page_number]  # 현재 페이지 가져오기
        
        # 페이지의 텍스트 추출 및 정리
        text = page.get_text().strip()
        # 페이지 번호와 텍스트를 딕셔너리 형태로 저장
        text_data.append({"response": text, "name": page_number + 1})
        
        # 현재 페이지에서 발견된 이미지 목록 가져오기
        images = page.get_images(full=True)

        # 페이지에서 찾은 모든 이미지를 순회하며 처리
        for image_index, img in enumerate(images, start=0):
            xref = img[0]  # 이미지의 XREF(참조 ID) 가져오기
            base_image = pdf_file.extract_image(xref)  # XREF를 사용해 이미지 추출
            image_bytes = base_image["image"]  # 추출한 이미지의 바이트 데이터
            image_ext = base_image["ext"]  # 이미지 파일 확장자 정보
            
            # 이미지 바이트 데이터를 PIL로 읽어와 이미지 파일로 저장
            image = Image.open(io.BytesIO(image_bytes))
            # 이미지 파일명에 페이지 번호와 이미지 인덱스 포함
            image.save(f"extracted_images/image_{page_number+1}_{image_index+1}.{image_ext}")


In [19]:
img_data

[]

In [20]:
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

### Image Captioning

In [21]:
for img in os.listdir("extracted_images"):
    # "extracted_images" 폴더에 있는 모든 이미지 파일을 순회합니다.
    image = Image.open(f"extracted_images/{img}")  # 현재 이미지 파일을 열어 Image 객체로 만듭니다.
    
    # 이미지와 요약 요청을 함께 전송하여 요약 생성 -> 이미지 to text하는 모델 
    response = model.generate_content([
        image,
        "You are an assistant tasked with summarizing tables, images and text for retrieval. "
        "These summaries will be embedded and used to retrieve the raw text or table elements. "
        "Give a concise summary of the table or text that is well optimized for retrieval. Table or text or image:"
    ])
    # response 객체에서 요약된 텍스트를 가져와 img_data 리스트에 저장
    img_data.append({
        "response": response.text,  # 생성된 요약 텍스트
        "name": img  # 원본 이미지 파일 이름
    })


In [22]:
img_data

[{'response': 'The table shows the prices of different types of 우도 민도 (Udo Mind) products. The prices are listed in Korean won. The types of Udo Mind products are: \n- \n- \n- \n-  . The table also notes that the prices may vary depending on the size and type of product.',
  'name': 'image_1_1.jpeg'}]

### Vectostore
- cohere embedding을 통해 추출한 이미지와 논문 내용을 임베딩하고 split하여 저장 

In [13]:
# Set embeddings
embedding_model = CohereEmbeddings(model="embed-english-v3.0", cohere_api_key="ew8pvCv9N3kLp07D1SXJHS1dH8DU13QQGsls7N7o")

# Load the document
docs_list = [Document(page_content=text['response'], metadata={"name": text['name']}) for text in text_data]
img_list = [Document(page_content=img['response'], metadata={"name": img['name']}) for img in img_data]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=400, chunk_overlap=50
)

doc_splits = text_splitter.split_documents(docs_list)
img_splits = text_splitter.split_documents(img_list)

In [14]:
# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits + img_splits, # adding the both text and image splits
    collection_name="multi_model_rag",
    embedding=embedding_model,
)

retriever = vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={'k': 1}, # number of documents to retrieve
            )

### Query

In [15]:
query = "What is the BLEU score of the Transformer (base model)?"

In [16]:
docs = retriever.invoke(query)

### Output

In [17]:
from langchain_core.output_parsers import StrOutputParser

# Prompt
system = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. 
Use three-to-five sentences maximum and keep the answer concise."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>"),
    ]
)

# LLM
llm = ChatCohere(model="command-r-plus", temperature=0)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"documents":docs[0].page_content, "question": query})
print(generation)

The Transformer (base model) achieves a BLEU score of 27.3.
